In [34]:
import numpy as np
import pandas as pd
import string

def hop_getGuo():
    #布尔数据
    #bool_data = pd.read_csv("input_data/bool438.csv")   
    #距离矩阵
    #dist_matrix = pd.read_csv("input_data/guo_distanceMatrix.csv")    
    #能量矩阵
    #energy_matrix = pd.read_csv("input_data/guo_energyMatrix.csv")
    
    #原始数据 从95开始是16C
    data = pd.read_csv("input_data/guo438.csv")
    
    #权重矩阵
    guo_weight = pd.read_csv("input_data/guo_weightMatrix.csv")
    guo_weight = guo_weight.values[:,1:]
    guo_weight = guo_weight.astype('float64')
    
    
    # 取出16-64的细胞及其在48个基因上的表达矩阵
    data = data.values[:,1:]
    data = data.astype('float64')
    lis = [8,28,38,58,108,178,428]
    data = data[lis]
    # 8,28,38,58,108,178,428
    
    
    #guo_weight.index = guo_weight.columns[1:]
    #weightGuo = guo_weight.loc[guo2names][guo2names].values
    
    return data,guo_weight

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import GetData
import pandas as pd
import validation as val

#(exp_data,init_weight,energy_data) = GetData.getSco2() 
exp_data,init_weight = hop_getGuo() 
#get GUO_data: GetData.getGuo()
(numGene, numData)= (np.shape(exp_data)[1],np.shape(exp_data)[0])


In [38]:
numData

7

In [108]:
from torch.nn import Module, Parameter

###############  Define a cell in RNN ###########
class ODECell(Module):
    def __init__(self, n_inputs, n_neurons,h):
        super(ODECell, self).__init__()
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        self.h = h
        
        ### self.Wx: Weight is a gene network, represented by a matrix,大小为input,neurons
        # 
        self.Wx = Parameter(torch.Tensor(n_inputs, n_neurons))
        
        ### self.diag:A diagnal matrix; 
        self.diag = Parameter(torch.Tensor(n_inputs))
        self.bias = Parameter(torch.Tensor(1,n_inputs))
        self.const =  Parameter(torch.Tensor(1,n_inputs))
        # 自定义的函数，用于初始化
        self.reset_parameters()
    
    #之前是初始化的问题
    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.n_neurons)
        for weight in self.parameters():
            weight.data.uniform_(-0.001, 0.001)
            
            
    def forward(self,X0):
        ###### Define ODE: ######
        Y0 = torch.mm(torch.tanh(X0), self.Wx)
        Y0 = Y0.mul(self.const)
        Y0 = Y0 - torch.mm(X0,torch.diag(self.diag)) + self.bias
        #Y0 = torch.tanh(torch.mm(X0, self.Wx)-torch.mm(X0,torch.diag(self.diag)))
        ### 
        Y1 = X0 +  self.h * Y0
        #Y1 = torch.tanh(Y1)
        return Y1




In [109]:
################ Define RNN ###################
import math
class ODERNN(nn.Module):
    def __init__(self, n_inputs, n_neurons,h):
        super(ODERNN, self).__init__()
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        self.h = h
        ### self.h: time interval in numerial ODE
        self.cell = ODECell(n_inputs, n_neurons,h)
        
        
    def forward(self, X0,steps):
        out = torch.empty(steps,self.n_inputs)
        nn.init.zeros_(out)
        out[0] = X0
        
        for i in range(steps-1):
            X0 = self.cell(X0)
            out[i+1] = X0
        return out


In [114]:
from torch.autograd import Variable

#### initialize parameters #####
#odernn.cell.Wx = Parameter(torch.from_numpy(init_weight).float())
#odernn.cell.diag = Parameter(torch.from_numpy(np.full((numGene),6)).float())
odernn = ODERNN(numGene,numGene,1)

EPOCH = 200000

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(odernn.parameters(),lr=0.001)

inputs = torch.tensor([exp_data[0]],dtype = torch.float)
datas = Variable(torch.from_numpy(exp_data))
datas = datas.float()

In [115]:
odernn.cell.Wx

Parameter containing:
tensor([[ 1.5582e-04, -9.2494e-04, -7.4032e-04,  ...,  5.8811e-04,
          4.2603e-04,  7.3910e-04],
        [ 9.1378e-04,  2.7320e-04,  1.9428e-04,  ..., -3.2818e-04,
         -2.3310e-04,  6.7948e-04],
        [ 5.2648e-05, -4.7650e-04, -1.8114e-04,  ..., -5.3971e-04,
          5.3513e-04,  7.8762e-04],
        ...,
        [-9.6928e-04, -5.1259e-04, -4.2113e-04,  ..., -4.1423e-04,
          8.0627e-04, -1.0572e-04],
        [ 9.7814e-04,  3.1331e-04, -1.3104e-04,  ..., -5.5724e-04,
          5.9849e-05,  4.1771e-04],
        [-4.5738e-04,  5.4247e-04, -8.0256e-04,  ..., -4.9244e-04,
         -5.8244e-05, -6.3667e-04]], requires_grad=True)

In [116]:
############### Training #####################
for epoch in range(EPOCH):    
    optimizer.zero_grad() 
    output = odernn(inputs,numData)
    ##forward(self, X0,steps):
    ## inputs：X0
    ## numData: steps
    loss = criterion(output,datas)
    loss.backward()                 # backpropagation, compute gradients
    optimizer.step()                # apply gradients
    if(epoch%500==0):
        print('Epoch: ', epoch, '| train loss: %.4f' % loss.item())

Epoch:  0 | train loss: 20.9751
Epoch:  500 | train loss: 7.5602
Epoch:  1000 | train loss: 7.2374
Epoch:  1500 | train loss: 6.6775
Epoch:  2000 | train loss: 6.0202
Epoch:  2500 | train loss: 5.4320
Epoch:  3000 | train loss: 4.9875
Epoch:  3500 | train loss: 4.6732
Epoch:  4000 | train loss: 4.4252
Epoch:  4500 | train loss: 4.2837
Epoch:  5000 | train loss: 4.2221
Epoch:  5500 | train loss: 4.1993
Epoch:  6000 | train loss: 4.1908
Epoch:  6500 | train loss: 4.1863
Epoch:  7000 | train loss: 4.1841
Epoch:  7500 | train loss: 4.1807
Epoch:  8000 | train loss: 4.1787
Epoch:  8500 | train loss: 4.1771
Epoch:  9000 | train loss: 4.1753
Epoch:  9500 | train loss: 4.1744
Epoch:  10000 | train loss: 4.1730
Epoch:  10500 | train loss: 4.1713
Epoch:  11000 | train loss: 4.1705
Epoch:  11500 | train loss: 4.1692
Epoch:  12000 | train loss: 4.1681
Epoch:  12500 | train loss: 4.1673
Epoch:  13000 | train loss: 4.1668
Epoch:  13500 | train loss: 4.1659
Epoch:  14000 | train loss: 4.1648
Epoch:  

Epoch:  117000 | train loss: 4.1343
Epoch:  117500 | train loss: 4.1344
Epoch:  118000 | train loss: 4.1342
Epoch:  118500 | train loss: 4.1344
Epoch:  119000 | train loss: 4.1351
Epoch:  119500 | train loss: 4.1341
Epoch:  120000 | train loss: 4.1342
Epoch:  120500 | train loss: 4.1347
Epoch:  121000 | train loss: 4.1368
Epoch:  121500 | train loss: 4.1381
Epoch:  122000 | train loss: 4.1347
Epoch:  122500 | train loss: 4.1434
Epoch:  123000 | train loss: 4.1352
Epoch:  123500 | train loss: 4.1346
Epoch:  124000 | train loss: 4.1339
Epoch:  124500 | train loss: 4.1433
Epoch:  125000 | train loss: 4.1347
Epoch:  125500 | train loss: 4.1341
Epoch:  126000 | train loss: 4.1386
Epoch:  126500 | train loss: 4.1423
Epoch:  127000 | train loss: 4.1364
Epoch:  127500 | train loss: 4.1341
Epoch:  128000 | train loss: 4.1352
Epoch:  128500 | train loss: 4.1411
Epoch:  129000 | train loss: 4.1372
Epoch:  129500 | train loss: 4.1373
Epoch:  130000 | train loss: 4.1351
Epoch:  130500 | train loss:

In [117]:
odernn.cell.Wx

Parameter containing:
tensor([[ 0.6061,  0.7783, -0.6002,  ..., -0.4806, -0.1160, -0.1285],
        [ 0.6069,  0.7795, -0.5993,  ..., -0.4815, -0.1167, -0.1285],
        [ 0.6060,  0.7788, -0.5996,  ..., -0.4817, -0.1159, -0.1284],
        ...,
        [ 0.6050,  0.7787, -0.5999,  ..., -0.4816, -0.1157, -0.1293],
        [ 0.6070,  0.7796, -0.5996,  ..., -0.4817, -0.1164, -0.1288],
        [ 0.6055,  0.7798, -0.6003,  ..., -0.4816, -0.1165, -0.1298]],
       requires_grad=True)

In [113]:
odernn.cell.Wx

Parameter containing:
tensor([[-0.2857, -0.4098,  0.3796,  ..., -0.1491, -0.0300,  0.0945],
        [-0.2867, -0.4096,  0.3797,  ..., -0.1496, -0.0308,  0.0938],
        [-0.2868, -0.4098,  0.3801,  ..., -0.1504, -0.0293,  0.0930],
        ...,
        [-0.2858, -0.4094,  0.3794,  ..., -0.1494, -0.0300,  0.0942],
        [-0.2865, -0.4111,  0.3804,  ..., -0.1485, -0.0293,  0.0936],
        [-0.2860, -0.4106,  0.3799,  ..., -0.1504, -0.0295,  0.0933]],
       requires_grad=True)

In [107]:
init_weight

array([[ 0.91672 ,  0.17323 ,  0.35824 , ..., -0.20776 , -0.1136  ,
         0.29931 ],
       [ 0.17323 ,  0.027752, -0.055472, ...,  0.68012 ,  0.62477 ,
        -0.66241 ],
       [ 0.35824 , -0.055472,  0.86062 , ..., -0.72143 , -0.29844 ,
         0.57733 ],
       ...,
       [-0.20776 ,  0.68012 , -0.72143 , ...,  0.49702 ,  0.21797 ,
        -0.39859 ],
       [-0.1136  ,  0.62477 , -0.29844 , ...,  0.21797 ,  0.54597 ,
        -0.33097 ],
       [ 0.29931 , -0.66241 ,  0.57733 , ..., -0.39859 , -0.33097 ,
         0.79015 ]])